# GroupfMRI13a - Automated SRActFlow using ANN-inspired representations
#### Using subject-specific FC
#### Using ActFlow

#### Takuya Ito
#### 05/29/2019

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import multiprocessing as mp
import scipy.stats as stats
import os
os.environ['OMP_NUM_THREADS'] = str(1)
import statsmodels.sandbox.stats.multicomp as mc
import seaborn as sns
import h5py
import tools_group_rsa as tools_group
import nibabel as nib
import EmpiricalSRActFlow_ANN_RSA_v2 as esr
sns.set_style("whitegrid")
plt.rcParams["font.family"] = "FreeSans"
import time


In [2]:
# Excluding 084
subjNums = ['013','014','016','017','018','021','023','024','026','027','028','030','031','032','033',
            '034','035','037','038','039','040','041','042','043','045','046','047','048','049','050',
            '053','055','056','057','058','062','063','066','067','068','069','070','072','074','075',
            '076','077','081','085','086','087','088','090','092','093','094','095','097','098','099',
            '101','102','103','104','105','106','108','109','110','111','112','114','115','117','119',
            '120','121','122','123','124','125','126','127','128','129','130','131','132','134','135',
            '136','137','138','139','140','141']



basedir = '/projects3/SRActFlow/'

# Using final partition
networkdef = np.loadtxt('/projects3/NetworkDiversity/data/network_partition.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = '/projects/AnalysisTools/ParcelsGlasser2016/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(xticks.keys())
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

# Load real motor response data

In [3]:
tools_group = reload(tools_group)
nResponses = 2
data_task_rh = np.zeros((len(glasser2),nResponses,len(subjNums)))
data_task_lh = np.zeros((len(glasser2),nResponses,len(subjNums)))

scount = 0
for subj in subjNums:
    data_task_rh[:,:,scount] = np.real(tools_group.loadMotorResponses(subj,hand='Right'))
    data_task_lh[:,:,scount] = np.real(tools_group.loadMotorResponses(subj,hand='Left'))
    scount += 1

tools_group_rsa.py:95: ComplexWarning: Casting complex values to real discards the imaginary part
  tmpdat[:,0] = data[:,2]
tools_group_rsa.py:96: ComplexWarning: Casting complex values to real discards the imaginary part
  tmpdat[:,1] = data[:,3]
tools_group_rsa.py:98: ComplexWarning: Casting complex values to real discards the imaginary part
  tmpdat[:,0] = data[:,0] #lmid
tools_group_rsa.py:99: ComplexWarning: Casting complex values to real discards the imaginary part
  tmpdat[:,1] = data[:,1] #lind


# Define wrapper function for SRActFlow

In [33]:
esr = reload(esr)
tools_group = reload(tools_group)
def subjSRActFlow_PCFC((subj,seed)):
    print 'Subject', subj
    
    #### LoadFC
    inputtypes = ['color','ori','pitch','constant']
    inputkeys = ['RED','VERTICAL','HIGH','CONSTANT']
    fc_input2hidden = {}
    eig_input2hidden = {}
    i = 0
    for inputtype in inputtypes:
        fc_input2hidden[inputkeys[i]], fc_hidden2motorresp, eig_input2hidden[inputkeys[i]], eig_hidden2motorresp = tools_group.loadSubjActFlowFC(subj,inputtype,pc_space=True)
        i += 1

    # Load rules to hidden FC mappings
    fc_logic2hidden, fc_hidden2motorresp, eig_logic2hidden, eig_hidden2motorresp = tools_group.loadSubjActFlowFC(subj,'Logic',pc_space=True)
    fc_sensory2hidden, fc_hidden2motorresp, eig_sensory2hidden, eig_hidden2motorresp = tools_group.loadSubjActFlowFC(subj,'Sensory',pc_space=True)
    fc_motor2hidden, fc_hidden2motorresp, eig_motor2hidden, eig_hidden2motorresp = tools_group.loadSubjActFlowFC(subj,'Motor',pc_space=True)
    
    #### Run Empirical SRActFlow
    obj = esr.EmpiricalActFlow(subj)
    # Input
    obj.fc_input2hidden = fc_input2hidden
    obj.eig_input2hidden = eig_input2hidden
    # Rules
    obj.fc_logic2hidden = fc_logic2hidden
    obj.eig_logic2hidden = eig_logic2hidden
    obj.fc_sensory2hidden = fc_sensory2hidden
    obj.eig_sensory2hidden = eig_sensory2hidden
    obj.fc_motor2hidden = fc_motor2hidden
    obj.eig_motor2hidden = eig_motor2hidden
    # hidden 2 motor
    obj.fc_hidden2motorresp = fc_hidden2motorresp
    obj.eig_hidden2motorresp = eig_hidden2motorresp
    
    obj.extractAllActivations(seed,n_inputs=1)
    

    actflow = obj.generateActFlowPredictions_PCFC(thresh=0,verbose=False)
    del obj
    return actflow

global fc_input2hidden
global fc_logic2hidden
global fc_sensory2hidden
global fc_motor2hidden

inputs = []
for i in range(len(subjNums)):
    inputs.append((subjNums[i],np.random.randint(1000000)))

timestart = time.time()
pool = mp.Pool(processes=2)
results = pool.map_async(subjSRActFlow_PCFC,inputs).get()
pool.close()
pool.join()
timeend = time.time()
print "time elapsed:", timeend-timestart

actflow_predictions = np.asarray(results)

Subject 013
Subject 031
Subject 032
Subject 014
Subject 033
Subject 016
Subject 034
Subject 017
Subject 035
Subject 018
Subject 037
Subject 021
Subject 038
Subject 023
Subject 039
Subject 024
Subject 040
Subject 026
Subject 041
Subject 027
Subject 042
Subject 028
Subject 043
Subject 030
Subject 045
Subject 063
Subject 046
Subject 066
Subject 047
Subject 067
Subject 048
Subject 068
Subject 049
Subject 069
Subject 050
Subject 070
Subject 053
Subject 072
Subject 055
Subject 074
Subject 056
Subject 075
Subject 057
Subject 076
Subject 058
Subject 077
Subject 062
Subject 081
Subject 085
Subject 101
Subject 086
Subject 102
Subject 087
Subject 103
Subject 088
Subject 104
Subject 090
Subject 105
Subject 092
Subject 106
Subject 093
Subject 108
Subject 094
Subject 109
Subject 095
Subject 110
Subject 097
Subject 111
Subject 098
Subject 112
Subject 099
Subject 114
Subject 115
Subject 129
Subject 117
Subject 130
Subject 119
Subject 131
Subject 120
Subject 132
Subject 121
Subject 134
Subject 122
Subj

#### Identify target vertices

In [34]:
# Set indices for layer-by-layer vertices
targetdir = '/projects3/SRActFlow/data/results/GroupfMRI/MotorResponseDecoding/'
motor_resp_regions_LH = np.loadtxt(targetdir + 'MotorResponseRegions_LH.csv',delimiter=',')
motor_resp_regions_RH = np.loadtxt(targetdir + 'MotorResponseRegions_RH.csv',delimiter=',')
targetROIs = np.hstack((motor_resp_regions_LH,motor_resp_regions_RH))

target_ind = []
for roi in targetROIs:
    roi_ind = np.where(glasser2==roi+1)[0]
    target_ind.extend(roi_ind)
target_ind = np.asarray(target_ind)

#### Compute average activity for each response, for each subject

In [35]:
scount = 0
actflow_rh = np.zeros(data_task_rh.shape)
actflow_lh = np.zeros(data_task_lh.shape)
for scount in range(len(subjNums)):
    # RMID
    actflow_rh[target_ind,0,scount] = actflow_predictions[scount,:,2]
    # RIND
    actflow_rh[target_ind,1,scount] = actflow_predictions[scount,:,3]
    # LMID
    actflow_lh[target_ind,0,scount] = actflow_predictions[scount,:,0]
    # LIND
    actflow_lh[target_ind,1,scount] = actflow_predictions[scount,:,1]


# Run across subject decoding on right-hand motor responses

#### Run decoding

In [36]:
tools_group = reload(tools_group)
nproc = 20
nResponses = 2
ncvs = 1

rois = np.asarray([8,52,9])-1
# rois = np.asarray([52])-1
# rois = np.where(networkdef==networkmappings['smn'])[0]
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])

# realdata = stats.zscore(data_task_rh[target_ind,:,:],axis=0).copy()
# flowdata = stats.zscore(actflow_rh[target_ind,:,:],axis=0).copy()
realdata = data_task_rh[roi_ind,:,:].copy()
flowdata = actflow_rh[roi_ind,:,:].copy()


distances_baseline_rh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_rh[0,:],rmatch,rmismatch = tools_group.actflowDecodings(realdata,
                                                          flowdata,effects=True,
                                                          ncvs=ncvs, nproc=nproc)

#### Compute statistics

In [37]:
statistics_rh = np.zeros((distances_baseline_rh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_rh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_rh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_rh[0,0] = np.mean(distances_baseline_rh[0,:])
statistics_rh[0,1] = p

print 'Activity flow accuracy =', statistics_rh[0,0]
print 'p =', statistics_rh[0,1]
print 'Matched spatial correlation:', np.mean(rmatch)
print 'Mismatched spatial correlation:', np.mean(rmismatch)



Activity flow accuracy = 0.427083333333
p = 0.974467622661
Matched spatial correlation: -0.00816645604295
Mismatched spatial correlation: 0.00816645604295


# Run across subject decoding on left-hand motor responses

#### Run decoding

In [38]:
tools_group = reload(tools_group)
nproc = 20
ncvs = 1

rois = np.asarray([188,189,232]) - 1
# rois = np.asarray([189]) - 1
# rois = np.where(networkdef==networkmappings['smn'])[0]
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])
    
# realdata = stats.zscore(data_task_lh[roi_ind,:,:],axis=0).copy()
# flowdata = stats.zscore(actflow_lh[roi_ind,:,:],axis=0).copy()
realdata = data_task_lh[roi_ind,:,:].copy()
flowdata = actflow_lh[roi_ind,:,:].copy()


distances_baseline_lh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_lh[0,:],rmatch,rmismatch = tools_group.actflowDecodings(realdata,
                                                          flowdata,effects=True,
                                                          ncvs=ncvs, nproc=nproc)
    

#### Compute statistics

In [39]:
statistics_lh = np.zeros((distances_baseline_lh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_lh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_lh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_lh[0,0] = np.mean(distances_baseline_lh[0,:])
statistics_lh[0,1] = p

print 'Activity flow accuracy =', statistics_lh[0,0]
print 'p =', statistics_lh[0,1]
print 'Matched spatial correlation:', np.mean(rmatch)
print 'Mismatched spatial correlation:', np.mean(rmismatch)



Activity flow accuracy = 0.458333333333
p = 0.860509115482
Matched spatial correlation: -0.0107879767516
Mismatched spatial correlation: 0.0107879767516


___

# Control -- Don't apply a threshold in the hidden layers

#### Define wrapper function for Control - SRActFlow

In [26]:
esr = reload(esr)
tools_group = reload(tools_group)
def subjSRActFlow_PCFC_ControlNoThresh((subj,seed)):
    print 'Subject', subj
    
    #### LoadFC
    inputtypes = ['color','ori','pitch','constant']
    inputkeys = ['RED','VERTICAL','HIGH','CONSTANT']
    fc_input2hidden = {}
    eig_input2hidden = {}
    i = 0
    for inputtype in inputtypes:
        fc_input2hidden[inputkeys[i]], fc_hidden2motorresp, eig_input2hidden[inputkeys[i]], eig_hidden2motorresp = tools_group.loadSubjActFlowFC(subj,inputtype,pc_space=True)
        i += 1

    # Load rules to hidden FC mappings
    fc_logic2hidden, fc_hidden2motorresp, eig_logic2hidden, eig_hidden2motorresp = tools_group.loadSubjActFlowFC(subj,'Logic',pc_space=True)
    fc_sensory2hidden, fc_hidden2motorresp, eig_sensory2hidden, eig_hidden2motorresp = tools_group.loadSubjActFlowFC(subj,'Sensory',pc_space=True)
    fc_motor2hidden, fc_hidden2motorresp, eig_motor2hidden, eig_hidden2motorresp = tools_group.loadSubjActFlowFC(subj,'Motor',pc_space=True)
    
    #### Run Empirical SRActFlow
    obj = esr.EmpiricalActFlow(subj)
    # Input
    obj.fc_input2hidden = fc_input2hidden
    obj.eig_input2hidden = eig_input2hidden
    # Rules
    obj.fc_logic2hidden = fc_logic2hidden
    obj.eig_logic2hidden = eig_logic2hidden
    obj.fc_sensory2hidden = fc_sensory2hidden
    obj.eig_sensory2hidden = eig_sensory2hidden
    obj.fc_motor2hidden = fc_motor2hidden
    obj.eig_motor2hidden = eig_motor2hidden
    # hidden 2 motor
    obj.fc_hidden2motorresp = fc_hidden2motorresp
    obj.eig_hidden2motorresp = eig_hidden2motorresp
    
    obj.extractAllActivations(seed,n_inputs=40)
    

    actflow = obj.generateActFlowPredictions_PCFC(thresh=None,verbose=False)
    del obj
    return actflow

global fc_input2hidden
global fc_logic2hidden
global fc_sensory2hidden
global fc_motor2hidden

inputs = []
for i in range(len(subjNums)):
    inputs.append((subjNums[i],np.random.randint(1000000)))

timestart = time.time()
pool = mp.Pool(processes=2)
results = pool.map_async(subjSRActFlow_PCFC_ControlNoThresh,inputs).get()
pool.close()
pool.join()
timeend = time.time()
print "time elapsed:", timeend-timestart

actflow_predictions_control = np.asarray(results)

Subject 013
Subject 031
Subject 032
Subject 014
Subject 033
Subject 016
Subject 034
Subject 017
Subject 035
Subject 018
Subject 037
Subject 021
Subject 038
Subject 023
Subject 039
Subject 024
Subject 040
Subject 026
Subject 041
Subject 027
Subject 042
Subject 028
Subject 043
Subject 030
Subject 045
Subject 063
Subject 046
Subject 066
Subject 047
Subject 067
Subject 048
Subject 068
Subject 049
Subject 069
Subject 050
Subject 070
Subject 053
Subject 072
Subject 055
Subject 074
Subject 056
Subject 075
Subject 057
Subject 076
Subject 058
Subject 077
Subject 062
Subject 081
Subject 085
Subject 101
Subject 086
Subject 102
Subject 087
Subject 103
Subject 088
Subject 104
Subject 090
Subject 105
Subject 092
Subject 106
Subject 093
Subject 108
Subject 094
Subject 109
Subject 095
Subject 110
Subject 097
Subject 111
Subject 098
Subject 112
Subject 099
Subject 114
Subject 115
Subject 129
Subject 117
Subject 130
Subject 119
Subject 131
Subject 120
Subject 132
Subject 121
Subject 134
Subject 122
Subj

#### Identify target vertices

In [27]:
# Set indices for layer-by-layer vertices
targetdir = '/projects3/SRActFlow/data/results/GroupfMRI/MotorResponseDecoding/'
motor_resp_regions_LH = np.loadtxt(targetdir + 'MotorResponseRegions_LH.csv',delimiter=',')
motor_resp_regions_RH = np.loadtxt(targetdir + 'MotorResponseRegions_RH.csv',delimiter=',')
targetROIs = np.hstack((motor_resp_regions_LH,motor_resp_regions_RH))

target_ind = []
for roi in targetROIs:
    roi_ind = np.where(glasser2==roi+1)[0]
    target_ind.extend(roi_ind)
target_ind = np.asarray(target_ind)

#### Compute average activity for each response, for each subject

In [28]:
scount = 0
actflow_rh = np.zeros(data_task_rh.shape)
actflow_lh = np.zeros(data_task_lh.shape)
for scount in range(len(subjNums)):
    # RMID
    actflow_rh[target_ind,0,scount] = actflow_predictions_control[scount,:,2]
    # RIND
    actflow_rh[target_ind,1,scount] = actflow_predictions_control[scount,:,3]
    # LMID
    actflow_lh[target_ind,0,scount] = actflow_predictions_control[scount,:,0]
    # LIND
    actflow_lh[target_ind,1,scount] = actflow_predictions_control[scount,:,1]


# Run across subject decoding on right-hand motor responses -- no threshold

#### Run decoding

In [29]:
tools_group = reload(tools_group)
nproc = 20
nResponses = 2
ncvs = 1

rois = np.asarray([8,52,9])-1
# rois = np.asarray([52])-1
# rois = np.where(networkdef==networkmappings['smn'])[0]
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])

# realdata = stats.zscore(data_task_rh[target_ind,:,:],axis=0).copy()
# flowdata = stats.zscore(actflow_rh[target_ind,:,:],axis=0).copy()
realdata = data_task_rh[roi_ind,:,:].copy()
flowdata = actflow_rh[roi_ind,:,:].copy()


distances_baseline_rh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_rh[0,:],rmatch,rmismatch = tools_group.actflowDecodings(realdata,
                                                          flowdata,effects=True,
                                                          ncvs=ncvs, nproc=nproc)

#### Compute statistics

In [30]:
statistics_rh = np.zeros((distances_baseline_rh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_rh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_rh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_rh[0,0] = np.mean(distances_baseline_rh[0,:])
statistics_rh[0,1] = p

print 'Activity flow accuracy =', statistics_rh[0,0]
print 'p =', statistics_rh[0,1]
print 'Matched spatial correlation:', np.mean(rmatch)
print 'Mismatched spatial correlation:', np.mean(rmismatch)



Activity flow accuracy = 0.46875
p = 0.786322705302
Matched spatial correlation: -0.0045094293752
Mismatched spatial correlation: -0.00260183079112


# Run across subject decoding on left-hand motor responses -- no threshold

#### Run decoding

In [31]:
tools_group = reload(tools_group)
nproc = 20
ncvs = 1

rois = np.asarray([188,189,232]) - 1
# rois = np.asarray([189]) - 1
# rois = np.where(networkdef==networkmappings['smn'])[0]
roi_ind = []
for roi in rois:
    roi_ind.extend(np.where(glasser2==roi+1)[0])
    
# realdata = stats.zscore(data_task_lh[roi_ind,:,:],axis=0).copy()
# flowdata = stats.zscore(actflow_lh[roi_ind,:,:],axis=0).copy()
realdata = data_task_lh[roi_ind,:,:].copy()
flowdata = actflow_lh[roi_ind,:,:].copy()


distances_baseline_lh = np.zeros((1,len(subjNums)*nResponses))
distances_baseline_lh[0,:],rmatch,rmismatch = tools_group.actflowDecodings(realdata,
                                                          flowdata,effects=True,
                                                          ncvs=ncvs, nproc=nproc)
    

#### Compute statistics

In [32]:
statistics_lh = np.zeros((distances_baseline_lh.shape[0],2))
ntrials = len(subjNums)*2
p = stats.binom_test(np.mean(distances_baseline_lh[0,:])*ntrials,n=ntrials,p=0.5)
if np.mean(distances_baseline_lh[0,:])>0.5:
    p = p/2.0
else:
    p = 1.0-p/2.0

    
statistics_lh[0,0] = np.mean(distances_baseline_lh[0,:])
statistics_lh[0,1] = p

print 'Activity flow accuracy =', statistics_lh[0,0]
print 'p =', statistics_lh[0,1]
print 'Matched spatial correlation:', np.mean(rmatch)
print 'Mismatched spatial correlation:', np.mean(rmismatch)



Activity flow accuracy = 0.557291666667
p = 0.0647094657579
Matched spatial correlation: 0.0117624505383
Mismatched spatial correlation: 0.00801956404574
